# 1. Preprocessing 

## 1.1 Requirements

In [1]:
pip3 install pandas
pip install --upgrade nltk
pip install stanza

## 1.2 Imports

In [1]:
import pandas as pd
import re
import string
import nltk
nltk.download('stopwords')
#import stanza
#stanza.download('pt')

[nltk_data] Downloading package stopwords to /home/alina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Table only with news 'headline' and 'content'

pd.set_option('max_colwidth',150)
data = pd.read_csv ('data.csv')
data_text = data.loc[:, ['headline', 'content']]

In [3]:
# Search for null values

for col in data_text.columns:
    print(col, data_text[col].isnull().sum())

headline 11
content 0


In [4]:
# Join 'headline' and 'content'

for index, row in data_text.iterrows():
    if isinstance(row['headline'], float): pass
    else: 
        if row['headline'] not in row['content']: row['content'] = row['headline'] + ' ' + row['content']
data_text.drop('headline', axis = 1, inplace = True)
data_text

,content
0,"Prisão perpétua para homem que tentou assassinar senadora No dia 8 de Janeiro de 2011 Jared Loughner tentou matar, sem sucesso, a então senadora G..."
1,"John Nash, matemático de Uma Mente Brilhante, morre em acidente John Nash, o matemático e Nobel da Economia que foi retratado por Russell Crowe no..."
2,O mito da reeleição mínima garantida Cavaco sairá desta campanha pior do que entrou. Os casos das acções na SLN ou da casa de férias no Algarve nã...
3,"Morreu Rita Levi-Montalcini, a grande dama da ciência italiana A cientista e senadora italiana Rita Levi-Montalcini, Prémio Nobel da Medicina de 1..."
4,"Por trás da porta amarela, há um homem com problemas psicológicos Pub Ninguém sabe o que ele está a fazer. Ninguém sabe em que é que pensa, o que..."
...,...
654,Ao terceiro dia Hendrix ressuscitou Festival de Curtas Metragens recordou último concerto do guitarrista em Inglaterra Festival de Curtas Metrage...
655,Capital acordou tarde e tensa votação Jornada eleitoral decorreu com civismo e ansiedade Escaramuças fazem um morto e um ferido Jornada eleitoral...
656,"- out3022 Arguidos começam hoje a discutir as provas casa pia Rui Teixeira abandona processo de pedofilia, que será sorteado a outro magistrado..."
657,"""Nunca me deixei ser empurrado"" Alexandra Marques Ciente de que só passará à segunda volta ""se uma grande parte do PS"" votar nele, diz que Sócra..."


In [5]:
# Cleaning

irrelevant = ["pub", "pág", "reportagem", "clique", "subscrever", "partilhar", "dn", "publico.pt", "rtp", "cartaz", "tv", "hoje", "amanhã", "ontem", "segundafeira", "terçafeira", "quartafeira", "quintafeira", "sextafeira", "sábado", "domingo", "janeiro", "fevereiro", "março", "abril", "maio", "junho", "julho", "agosto", "outubro", "novembro", "dezembro"] # list with irrelevant words/expressions do the news content

stopwords = nltk.corpus.stopwords.words('portuguese') # pt stopwords

#nlp = stanza.Pipeline('pt') # pt stanza

def clean_text(text):

    text = text.lower() # lowercase
    text = re.sub(r'http\S+', '', text) # URLs removal
    text = re.sub('\[.*?\]', '', text) # remove text in square brackets
    text = re.sub(r"\([^()]*\)", '', text) # remove text within parentheses
    text = re.sub(r'-(me|te|se|nos|vos|o|os|a|as|lo|los|la|las|lhe|lhes|lha|lhas|lho|lhos|no|na|nas|mo|ma|mos|mas|to|ta|tos|tas)', '', text) # remove pronouns at the end of the wo
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # remove punctuation
    text = re.sub('[‘’“”«|»…]', '', text)
    text = re.sub('\w*\d\w*', '', text) # remove words containing numbers
    text = ' '.join([word for word in text.split() if word not in stopwords]) # remove stopwords
    text = re.sub(r'Pág.*por pág$', '', text) # remove irrelevant text
    text = ' '.join([word for word in text.split() if word not in irrelevant]) 
    #text = nlp(text)
    #text = ' '.join([word.lemma for sent in text.sentences for word in sent.words]) 

    return text

data_clean = pd.DataFrame(data_text.content.apply(lambda x: clean_text(x)))
data_clean

,content
0,prisão perpétua homem tentou assassinar senadora dia jared loughner tentou matar sucesso então senadora gabrielle giffords eleita partido democrat...
1,john nash matemático mente brilhante morre acidente john nash matemático nobel economia retratado russell crowe filme mente brilhante morreu acide...
2,mito reeleição mínima garantida cavaco sairá desta campanha pior entrou casos acções sln casa férias algarve revelaram qualquer ilegalidade pudess...
3,morreu rita levintalcini grande dama ciência italiana cientista senadora italiana rita levintalcini prémio nobel medicina morreu neste anos casa r...
4,trás porta amarela homem problemas psicológicos ninguém sabe fazer ninguém sabe pensa come sobrevive dois anos desde mulher pânico levou dois filh...
...,...
654,terceiro dia hendrix ressuscitou festival curtas metragens recordou último concerto guitarrista inglaterra festival curtas metragens recordou últi...
655,capital acordou tarde tensa votação jornada eleitoral decorreu civismo ansiedade escaramuças fazem morto ferido jornada eleitoral decorreu civismo...
656,arguidos começam discutir provas casa pia rui teixeira abandona processo pedofilia sorteado outro magistradocruz abrantes tentam descredibilizar a...
657,nunca deixei ser empurrado alexandra marques ciente passará segunda volta grande parte ps votar nele diz sócrates sossego belém outros candidatos ...


In [6]:
# Word tokenization

def tokenize(text):

    text = text.split()

    return text

data_tokenized = pd.DataFrame(data_clean.content.apply(lambda x: tokenize(x)))
data_tokenized

,content
0,"[prisão, perpétua, homem, tentou, assassinar, senadora, dia, jared, loughner, tentou, matar, sucesso, então, senadora, gabrielle, giffords, eleita..."
1,"[john, nash, matemático, mente, brilhante, morre, acidente, john, nash, matemático, nobel, economia, retratado, russell, crowe, filme, mente, bril..."
2,"[mito, reeleição, mínima, garantida, cavaco, sairá, desta, campanha, pior, entrou, casos, acções, sln, casa, férias, algarve, revelaram, qualquer,..."
3,"[morreu, rita, levintalcini, grande, dama, ciência, italiana, cientista, senadora, italiana, rita, levintalcini, prémio, nobel, medicina, morreu, ..."
4,"[trás, porta, amarela, homem, problemas, psicológicos, ninguém, sabe, fazer, ninguém, sabe, pensa, come, sobrevive, dois, anos, desde, mulher, pân..."
...,...
654,"[terceiro, dia, hendrix, ressuscitou, festival, curtas, metragens, recordou, último, concerto, guitarrista, inglaterra, festival, curtas, metragen..."
655,"[capital, acordou, tarde, tensa, votação, jornada, eleitoral, decorreu, civismo, ansiedade, escaramuças, fazem, morto, ferido, jornada, eleitoral,..."
656,"[arguidos, começam, discutir, provas, casa, pia, rui, teixeira, abandona, processo, pedofilia, sorteado, outro, magistradocruz, abrantes, tentam, ..."
657,"[nunca, deixei, ser, empurrado, alexandra, marques, ciente, passará, segunda, volta, grande, parte, ps, votar, nele, diz, sócrates, sossego, belém..."


In [7]:
# Write to csv

data_tokenized.to_csv("data_cleaned.csv")